# Introduction

The notebook is intended to develop a character-based LLM trained on [Shakespeare Literature](https://github.com/karpathy/ng-video-lecture/blob/master/input.txt).

**Resources**
- [Reference tutorial from Andrej Karpathy](https://www.youtube.com/watch?v=kCc8FmEb1nY)

In [2]:
# Import Standard Libraries
import os

from pathlib import Path

# Read Data

In [3]:
# Define local data file path
train_data_file_path = Path(os.path.abspath('')).parents[1] / 'data' / 'character_based_llm_train_data.txt'

In [4]:
# Read data
with open(train_data_file_path, 'r', encoding='utf-8') as train_data_file:
    train_data = train_data_file.read()

In [7]:
# Define the vocabulary of characters in the train data
train_vocaulary = sorted(list(set(train_data)))
train_vocaulary_size = len(train_vocaulary)